<a href="https://colab.research.google.com/github/alessandrovicenti10/RAGonExcel/blob/main/exampleRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama_index
!pip install llama_parse


In [ ]:
import nest_asyncio

from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex
from IPython.display import Image, Markdown

from llama_parse import LlamaParse

from llama_index.core.node_parser import MarkdownElementNodeParser

In [ ]:
nest_asyncio.apply()

In [ ]:
import os

from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

llm_o1 = OpenAI(model="o1-mini")
llm_gpt4o_mini = OpenAI(model="gpt-4o-mini")
llm_o1_preview = OpenAI(model="o1-preview")

In [ ]:

from google.colab import userdata

parser = LlamaParse(
    api_key=userdata.get('LLAMA_CLOUD'),
    result_type="markdown",
)

documents = parser.load_data("/content/foglioICE_TABULAR.xlsx")

Started parsing the file under job_id 0425fc94-8ddc-488e-9b71-7383666e45be


In [ ]:
len(documents)

1

In [ ]:
print(documents[0].get_content())

In [ ]:
node_parser = MarkdownElementNodeParser(llm=llm_gpt4o_mini, num_workers=4)

In [ ]:
nodes = node_parser.get_nodes_from_documents(documents[:10])

1it [00:00, 187.10it/s]


In [ ]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

In [ ]:
len(nodes), len(base_nodes), len(objects)

(3, 1, 1)

In [ ]:
print(objects[0].get_content())

This table presents various types of steel products, their manufacturing processes, applications, and environmental impacts, including greenhouse gas emissions and end-of-life recovery rates, as reported in the Worldsteel 2023 study.,
with the following table title:
Steel Products and Their Environmental Impact,
with the following columns:
- Steel Type: Different categories of steel products.
- Manufacturing Process: Description of how each steel product is made.
- Applications: Common uses for each type of steel.
- GWP Impact: Greenhouse gas emissions impact per kg.
- EOL Recovery Rate: End-of-life recovery rate for each steel type.



In [ ]:
recursive_index = VectorStoreIndex(nodes=base_nodes + objects, llm=llm_gpt4o_mini)
recursive_query_engine_gpt4o_mini = recursive_index.as_query_engine(
    similarity_top_k=5, llm=llm_gpt4o_mini
)

In [ ]:
query = "How much CO2 is emitted to produce one kg of steel?"
response_recursive_gpt4o_mini = recursive_query_engine_gpt4o_mini.query(query)

In [ ]:
print("----------------------RESPONSE WITH GPT4O-MINI----------------------")
display(Markdown(f"{response_recursive_gpt4o_mini}"))

----------------------RESPONSE WITH GPT4O-MINI----------------------


The greenhouse gas emissions impact per kg of steel varies depending on the type of steel product. For example, the impact can range from approximately 1.07 kg CO2e to 1.61 kg CO2e per kg, depending on the specific manufacturing process and type of steel.

In [ ]:
query = "Analyzing the information you have from the excel file, tell me how much CO2 is emitted for the production of a water bottle made from 1 kg of aluminum"
response_recursive_gpt4o_mini = recursive_query_engine_gpt4o_mini.query(query)
print("----------------------RESPONSE WITH GPT4O-MINI----------------------")
display(Markdown(f"{response_recursive_gpt4o_mini}"))


----------------------RESPONSE WITH GPT4O-MINI----------------------


The provided information does not include specific data regarding the CO2 emissions associated with the production of aluminum or water bottles made from aluminum. Therefore, it is not possible to determine the CO2 emissions for the production of a water bottle made from 1 kg of aluminum based on the available context.